In [36]:
from erddapy import ERDDAP
import pandas as pd
import requests
import netCDF4 as nc

### ARGO ERDDAP server

In [12]:
# analysed endpoint: 
endpoint_url = "https://erddap.ifremer.fr/erddap"

#connect to server
argoERDDAP = ERDDAP(server=endpoint_url,protocol="tabledap")

### Analysis results:

- data is findable, not always accessible 
    - for some listed datasets: an additional step is needed to open the actual netCDF files 
    (though the documentation mentions this is "OpenDAP on top of NetCDF files" still extra step needed)
    - at moment of running the code, some datasets were not accessible 
    - 'log in' column in 'allDatasets' dataset indicates that not all data is freely available

- data granularity
    - some datasets (e.g. ArgaFloats) contain file paths in 'file' column --> retrieval of actual data requires additional steps
    - other datasets directly return measurement data
      

- semantics:
    - interoperability at higher level --> allDatasets dataset offers metadata about the datasets that are available through the ERDDAP server. Properties/columns used to describe this metadata are the same as in other ERDDAP servers (eg. the EMSO-ERIC ERDDAP server).  
    - other datasets offered by ERDDAP server contain different properties --> similar columns/properties but not defined through external standard terms
        - more prone towards wrongly combining different data  



**Exploration of the allDatasets dataset**  
~ retrieving a dataset that lists all available datasets on the ERDDAP server. The returned DataFrame will contain metadata for each dataset available on that server  
~ essentially a catalog of all datasets hosted on the server, including essential metadata that allows you to identify and filter the datasets of interest  

In [13]:
#Set a dataset ID
argoERDDAP.dataset_id = "allDatasets"

#Get data (as 2D dataframe)
argoERDDAP_allDatasets_df = argoERDDAP.to_pandas()

In [14]:
#explore columns
argoERDDAP_allDatasets_df.columns

Index(['datasetID', 'accessible', 'institution', 'dataStructure',
       'cdm_data_type', 'class', 'title', 'minLongitude (degrees_east)',
       'maxLongitude (degrees_east)', 'longitudeSpacing (degrees_east)',
       'minLatitude (degrees_north)', 'maxLatitude (degrees_north)',
       'latitudeSpacing (degrees_north)', 'minAltitude (m)', 'maxAltitude (m)',
       'minTime (UTC)', 'maxTime (UTC)', 'timeSpacing (seconds)', 'griddap',
       'subset', 'tabledap', 'MakeAGraph', 'sos', 'wcs', 'wms', 'files',
       'fgdc', 'iso19115', 'metadata', 'sourceUrl', 'infoUrl', 'rss', 'email',
       'testOutOfDate', 'outOfDate', 'summary'],
      dtype='object')

In [15]:
#view 
argoERDDAP_allDatasets_df

,datasetID,accessible,institution,dataStructure,cdm_data_type,class,title,minLongitude (degrees_east),maxLongitude (degrees_east),longitudeSpacing (degrees_east),...,fgdc,iso19115,metadata,sourceUrl,infoUrl,rss,email,testOutOfDate,outOfDate,summary
0,allDatasets,public,Ifremer,table,Other,EDDTableFromAllDatasets,* The List of All Active Datasets in this ERDD...,NaN,NaN,NaN,...,NaN,NaN,https://erddap.ifremer.fr/erddap/info/allDatas...,https://erddap.ifremer.fr/erddap,https://erddap.ifremer.fr/erddap,https://erddap.ifremer.fr/erddap/rss/allDatase...,https://erddap.ifremer.fr/erddap/subscriptions...,NaN,NaN,This dataset is a table which has a row of inf...
1,OACP-Argo-Global,public,LOPS/Ifremer,grid,Grid,EDDGridFromNcFiles,2000-2015 climatology of the Subtropical Mode ...,-242.156000,129.844000,1.000000,...,https://erddap.ifremer.fr/erddap/metadata/fgdc...,https://erddap.ifremer.fr/erddap/metadata/iso1...,https://erddap.ifremer.fr/erddap/info/OACP-Arg...,(local files),https://doi.org/10.17882/56503,https://erddap.ifremer.fr/erddap/rss/OACP-Argo...,https://erddap.ifremer.fr/erddap/subscriptions...,NaN,NaN,Maps of properties from OAC-P estimates. Therm...
2,ArgoFloats,public,Argo,table,TrajectoryProfile,EDDTableFromMultidimNcFiles,Argo Float Measurements,-179.999420,180.000000,NaN,...,https://erddap.ifremer.fr/erddap/metadata/fgdc...,https://erddap.ifremer.fr/erddap/metadata/iso1...,https://erddap.ifremer.fr/erddap/info/ArgoFloa...,(local files),https://argo.ucsd.edu/,https://erddap.ifremer.fr/erddap/rss/ArgoFloat...,https://erddap.ifremer.fr/erddap/subscriptions...,NaN,NaN,Argo float vertical profiles from Coriolis Glo...
3,ArgoFloats-synthetic-BGC,public,Argo,table,TrajectoryProfile,EDDTableFromMultidimNcFiles,Argo float synthetic vertical profiles : BGC data,-999.999000,181.706000,NaN,...,https://erddap.ifremer.fr/erddap/metadata/fgdc...,https://erddap.ifremer.fr/erddap/metadata/iso1...,https://erddap.ifremer.fr/erddap/info/ArgoFloa...,(local files),http://www.argodatamgt.org/Documentation,https://erddap.ifremer.fr/erddap/rss/ArgoFloat...,https://erddap.ifremer.fr/erddap/subscriptions...,now-5days,0.619229,Argo float synthetic vertical profiles : BGC data
4,ArgoFloats-reference,public,Argo,table,Other,EDDTableFromMultidimNcFiles,Argo Reference Measurements,-180.000000,179.999487,NaN,...,https://erddap.ifremer.fr/erddap/metadata/fgdc...,https://erddap.ifremer.fr/erddap/metadata/iso1...,https://erddap.ifremer.fr/erddap/info/ArgoFloa...,(local files),http://www.argodatamgt.org/DMQC/Reference-data...,https://erddap.ifremer.fr/erddap/rss/ArgoFloat...,https://erddap.ifremer.fr/erddap/subscriptions...,NaN,NaN,Argo float vertical profiles to be used in DMQ...
5,ArgoFloats-index,public,Argo,table,Other,EDDTableFromAsciiFiles,ArgoFloats index,-999.999000,346.813000,NaN,...,https://erddap.ifremer.fr/erddap/metadata/fgdc...,https://erddap.ifremer.fr/erddap/metadata/iso1...,https://erddap.ifremer.fr/erddap/info/ArgoFloa...,ftp://ftp.ifremer.fr/ifremer/argo/ar_index_glo...,http://www.argodatamgt.org/DMQC/Reference-data...,https://erddap.ifremer.fr/erddap/rss/ArgoFloat...,https://erddap.ifremer.fr/erddap/subscriptions...,NaN,NaN,Argo detailed index. Gathers data available at...
6,ArgoFloats-reference-CTD,log in,Argo,table,Other,EDDTableFromMultidimNcFiles,CTD Reference Measurements,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Conductivity, Temperature, Depth (CTD) Referen..."
7,SST_Anomalies_Caledonie,public,Jet Propulsion Laboratory,grid,Grid,EDDGridFromNcFiles,"Daily MUR SST, Final product",155.010000,175.010000,0.010000,...,https://erddap.ifremer.fr/erddap/metadata/fgdc...,https://erddap.ifremer.fr/erddap/metadata/iso1...,https://erddap.ifremer.fr/erddap/info/SST_Anom...,(local files),https://podaac.jpl.nasa.gov/ws/metadata/datase...,https://erddap.ifremer.fr/erddap/rss/SST_Anoma...,https://erddap.ifremer.fr/erddap/subscriptions...,now-109days,8.936340,"A merged, multi-sensor L4 Foundation Sea Surfa..."
8,OS_DYFAMED_1994-2014_D_TSO2

Exploration of **search information** for each data offered through the ERDDAP server

In [16]:
# get search information
searchinfo_df = pd.read_csv(argoERDDAP.get_search_url(response="csv"))

In [17]:
searchinfo_df.columns

Index(['griddap', 'Subset', 'tabledap', 'Make A Graph', 'wms', 'files',
       'Accessible', 'Title', 'Summary', 'FGDC', 'ISO 19115', 'Info',
       'Background Info', 'RSS', 'Email', 'Institution', 'Dataset ID'],
      dtype='object')

In [18]:
searchinfo_df

,griddap,Subset,tabledap,Make A Graph,wms,files,Accessible,Title,Summary,FGDC,ISO 19115,Info,Background Info,RSS,Email,Institution,Dataset ID
0,NaN,https://erddap.ifremer.fr/erddap/tabledap/allD...,https://erddap.ifremer.fr/erddap/tabledap/allD...,https://erddap.ifremer.fr/erddap/tabledap/allD...,NaN,NaN,public,* The List of All Active Datasets in this ERDD...,This dataset is a table which has a row of inf...,NaN,NaN,https://erddap.ifremer.fr/erddap/info/allDatas...,https://erddap.ifremer.fr/erddap,NaN,NaN,Ifremer,allDatasets
1,NaN,NaN,https://erddap.ifremer.fr/erddap/tabledap/Argo...,https://erddap.ifremer.fr/erddap/tabledap/Argo...,NaN,NaN,public,Argo Float Measurements,Argo float vertical profiles from Coriolis Glo...,https://erddap.ifremer.fr/erddap/metadata/fgdc...,https://erddap.ifremer.fr/erddap/metadata/iso1...,https://erddap.ifremer.fr/erddap/info/ArgoFloa...,https://argo.ucsd.edu/,https://erddap.ifremer.fr/erddap/rss/ArgoFloat...,https://erddap.ifremer.fr/erddap/subscriptions...,Argo,ArgoFloats
2,NaN,https://erddap.ifremer.fr/erddap/tabledap/Argo...,https://erddap.ifremer.fr/erddap/tabledap/Argo...,https://erddap.ifremer.fr/erddap/tabledap/Argo...,NaN,NaN,public,Argo float synthetic vertical profiles : BGC data,Argo float synthetic vertical profiles : BGC d...,https://erddap.ifremer.fr/erddap/metadata/fgdc...,https://erddap.ifremer.fr/erddap/metadata/iso1...,https://erddap.ifremer.fr/erddap/info/ArgoFloa...,http://www.argodatamgt.org/Documentation,https://erddap.ifremer.fr/erddap/rss/ArgoFloat...,https://erddap.ifremer.fr/erddap/subscriptions...,Argo,ArgoFloats-synthetic-BGC
3,NaN,NaN,https://erddap.ifremer.fr/erddap/tabledap/Argo...,https://erddap.ifremer.fr/erddap/tabledap/Argo...,NaN,NaN,public,Argo Reference Measurements,Argo float vertical profiles to be used in DMQ...,https://erddap.ifremer.fr/erddap/metadata/fgdc...,https://erddap.ifremer.fr/erddap/metadata/iso1...,https://erddap.ifremer.fr/erddap/info/ArgoFloa...,http://www.argodatamgt.org/DMQC/Reference-data...,https://erddap.ifremer.fr/erddap/rss/ArgoFloat...,https://erddap.ifremer.fr/erddap/subscriptions...,Argo,ArgoFloats-reference
4,NaN,https://erddap.ifremer.fr/erddap/tabledap/Argo...,https://erddap.ifremer.fr/erddap/tabledap/Argo...,https://erddap.ifremer.fr/erddap/tabledap/Argo...,NaN,NaN,public,ArgoFloats index,Argo detailed index. Gathers data available at...,https://erddap.ifremer.fr/erddap/metadata/fgdc...,https://erddap.ifremer.fr/erddap/metadata/iso1...,https://erddap.ifremer.fr/erddap/info/ArgoFloa...,http://www.argodatamgt.org/DMQC/Reference-data...,https://erddap.ifremer.fr/erddap/rss/ArgoFloat...,https://erddap.ifremer.fr/erddap/subscriptions...,Argo,ArgoFloats-index
5,NaN,NaN,NaN,NaN,NaN,NaN,log in,CTD Reference Measurements,"Conductivity, Temperature, Depth (CTD) Referen...",NaN,NaN,NaN,NaN,NaN,NaN,Argo,ArgoFloats-reference-CTD
6,NaN,https://erddap.ifremer.fr/erddap/tabledap/drif...,https://erddap.ifremer.fr/erddap/tabledap/drif...,https://erddap.ifremer.fr/erddap/tabledap/drif...,NaN,https://erddap.ifremer.fr/erddap/files/drifter...,public,Global Drifter Program - 1 Hour Interpolated Q...,Global Drifter Program hourly drifting buoy co...,https://erddap.ifremer.fr/erddap/metadata/fgdc...,https://erddap.ifremer.fr/erddap/metadata/iso1...,https://erddap.ifremer.fr/erddap/info/drifter_...,https://www.aoml.noaa.gov/phod/dac/dirall.html,https://erddap.ifremer.fr/erddap/rss/drifter_h...,https://erddap.ifremer.fr/erddap/subscriptions...,NOAA Atlantic Oceanographic and Meteorological...,drifter_hourly_qc
7,NaN,https://erddap.ifremer.fr/erddap/tabledap/drif...,https://erddap.ifremer.fr/erddap/tabledap/drif...,https://erddap.ifremer.fr/erddap/tabledap/drif...,NaN,https://erddap.ifremer.fr/erddap/files/drifter...,public,Global Drifter Program - 6 Hour Interpolated Q...,Global Drifter Program 6-hourly drifting buoy ...,https://erddap.ifremer.fr/erddap/metadata/fgdc...,https://erddap.ifremer.fr/erddap/metadata/iso1...,https://erddap.ifremer.fr/erddap/i

Exploration of **metadata information** available for each dataset offered through the ERDDAP server

In [19]:
# get metadata information for each dataset
metadatainfo_df = pd.DataFrame(columns=["Row Type", "Variable Name", "Attribute Name", "Data Type", "Value", "DatasetID"])
# get metadata for each dataset
for datasetID in argoERDDAP_allDatasets_df['datasetID']:
    argoERDDAP.dataset_id = datasetID
    try:
        _df = pd.read_csv(argoERDDAP.get_info_url(response="csv")) #metadata retrieved via info_url
        _df["DatasetID"] = datasetID
        metadatainfo_df = pd.concat([metadatainfo_df, _df], ignore_index=True)
    except Exception as e:
        print(f"there was an error for {datasetID}: '{e}'")
        continue

metadatainfo_df

there was an error for ArgoFloats-reference-CTD: 'HTTP Error 401: '


,Row Type,Variable Name,Attribute Name,Data Type,Value,DatasetID
0,attribute,NC_GLOBAL,cdm_data_type,String,Other,allDatasets
1,attribute,NC_GLOBAL,Conventions,String,"COARDS, CF-1.6, ACDD-1.3",allDatasets
2,attribute,NC_GLOBAL,creator_email,String,enzo.pauvy@ifremer.fr,allDatasets
3,attribute,NC_GLOBAL,creator_name,String,Sismer-helpdesk,allDatasets
4,attribute,NC_GLOBAL,creator_url,String,https://erddap.ifremer.fr/erddap,allDatasets
...,...,...,...,...,...,...
9976,attribute,elevation,sdn_parameter_urn,String,SDN:P01::ALATZZ01,gebco2021
9977,attribute,elevation,sdn_uom_name,String,Metres,gebco2021
9978,attribute,elevation,sdn_uom_urn,String,SDN:P06::ULAA,gebco2021
9979,attribute,elevation,standard_name,String,height_above_mean_sea_level,gebco2021


In [20]:
# explore returned metadata
metadatainfo_df.groupby(['DatasetID', 'Row Type']).nunique()

Variable Name  Attribute Name  Data Type  \
DatasetID            Row Type                                              
ArgoFloats           attribute             60              52          4   
                     variable              59               0          4   
ArgoFloats-index     attribute              9              32          4   
                     variable               8               0          4   
ArgoFloats-reference attribute             13              38          4   
...                                       ...             ...        ...   
histo_coord          dimension              3               0          2   
                     variable              24               0          1   
traj_lonlat_init_t0  attribute              7              28          5   
                     dimension              2               0          1   
                     variable               4               0          1   

                                Value  
DatasetID            Row Type          
ArgoFloats           attribute    174  
                     variable       0  
ArgoFloats-index     attribute     45  
                     variable       0  
ArgoFloats-reference attribute     61  
...                               ...  
histo_coord          dimension      3  
                     variable       1  
traj_lonlat_init_t0  attribute     39  
                     dimension      2  
                     variable       1  

[139 rows x 4 columns]

In [21]:
# Row Type = 'dimension'
metadatainfo_df[ metadatainfo_df['Row Type'] == 'dimension'].groupby(['DatasetID', 'Variable Name']).nunique()

Row Type  Attribute Name  Data Type  Value
DatasetID           Variable Name                                            
HF_75c7_5b60_95d8   depth                 1               0          1      1
                    latitude              1               0          1      1
                    longitude             1               0          1      1
                    time                  1               0          1      1
HF_ac49_84ad_3eb6   depth                 1               0          1      1
...                                     ...             ...        ...    ...
histo_coord         latitude              1               0          1      1
                    longitude             1               0          1      1
                    time                  1               0          1      1
traj_lonlat_init_t0 frame                 1               0          1      1
                    ntraj                 1               0          1      1

[130 rows x 4 columns]

In [22]:
# Row Type = 'variable'
#note: in netCDFs, attributes represent additional information 
metadatainfo_df[ metadatainfo_df['Row Type'] == 'variable'].groupby(['DatasetID', 'Variable Name']).nunique()

Row Type  Attribute Name  \
DatasetID           Variable Name                                           
ArgoFloats          chla                                1               0   
                    chla_qc                             1               0   
                    config_mission_number               1               0   
                    cycle_number                        1               0   
                    data_center                         1               0   
...                                                   ...             ...   
histo_coord         hist_stop_coast_TELUK_BENOA         1               0   
traj_lonlat_init_t0 traj_iU_init_t0                     1               0   
                    traj_jV_init_t0                     1               0   
                    traj_lat_init_t0                    1               0   
                    traj_lon_init_t0                    1               0   

                                                 Data Type  Value  
DatasetID           Variable Name                                  
ArgoFloats          chla                                 1      0  
                    chla_qc                              1      0  
                    config_mission_number                1      0  
                    cycle_number                         1      0  
                    data_center                          1      0  
...                                                    ...    ...  
histo_coord         hist_stop_coast_TELUK_BENOA          1      1  
traj_lonlat_init_t0 traj_iU_init_t0                      1      1  
                    traj_jV_init_t0                      1      1  
                    traj_lat_init_t0                     1      1  
                    traj_lon_init_t0                     1      1  

[963 rows x 4 columns]

In [23]:
# Row Type = 'attribute'
metadatainfo_df[ metadatainfo_df['Row Type'] == 'attribute'].groupby(['DatasetID', 'Attribute Name']).nunique()

Row Type  Variable Name  \
DatasetID           Attribute Name                                      
ArgoFloats          C_format                         1              9   
                    Conventions                      1              1   
                    Easternmost_Easting              1              1   
                    FORTRAN_format                   1              9   
                    Northernmost_Northing            1              1   
...                                                ...            ...   
traj_lonlat_init_t0 standard_name                    1              4   
                    standard_name_vocabulary         1              1   
                    summary                          1              1   
                    title                            1              1   
                    units                            1              4   

                                              Data Type  Value  
DatasetID           Attribute Name                              
ArgoFloats          C_format                          1      2  
                    Conventions                       1      1  
                    Easternmost_Easting               1      1  
                    FORTRAN_format                    1      2  
                    Northernmost_Northing             1      1  
...                                                 ...    ...  
traj_lonlat_init_t0 standard_name                     1      4  
                    standard_name_vocabulary          1      1  
                    summary                           1      1  
                    title                             1      1  
                    units                             1      3  

[3307 rows x 4 columns]

In [24]:
# attributes and variables/dimensions with which they're associated
metadatainfo_df[ metadatainfo_df['Row Type'] == 'attribute'].groupby(['DatasetID', 'Attribute Name', 'Variable Name']).nunique()

Row Type  Data Type  \
DatasetID           Attribute Name Variable Name                              
ArgoFloats          C_format       pres                        1          1   
                                   pres_adjusted               1          1   
                                   pres_adjusted_error         1          1   
                                   psal                        1          1   
                                   psal_adjusted               1          1   
...                                                          ...        ...   
traj_lonlat_init_t0 title          NC_GLOBAL                   1          1   
                    units          frame                       1          1   
                                   ntraj                       1          1   
                                   traj_lat_init_t0            1          1   
                                   traj_lon_init_t0            1          1   

                                                        Value  
DatasetID           Attribute Name Variable Name               
ArgoFloats          C_format       pres                     1  
                                   pres_adjusted            1  
                                   pres_adjusted_error      1  
                                   psal                     1  
                                   psal_adjusted            1  
...                                                       ...  
traj_lonlat_init_t0 title          NC_GLOBAL                1  
                    units          frame                    1  
                                   ntraj                    1  
                                   traj_lat_init_t0         1  
                                   traj_lon_init_t0         1  

[8888 rows x 3 columns]

Exploration of data from each dataset offered through the ERDDAP server

In [45]:
#list datasets
print(f"there are {len(argoERDDAP_allDatasets_df['datasetID'].drop_duplicates())} datasets available via ARGO ERDDAP")

# Connect to server 
erddap = ERDDAP(server="https://erddap.ifremer.fr/erddap", protocol="tabledap")

# Set a reasonable timeout value (e.g., 10 seconds)
timeout_seconds = 10

for datasetID in argoERDDAP_allDatasets_df['datasetID']:
    print(f"Processing dataset: {datasetID}")
    
    if datasetID not in ['ArgoFloats', 'ArgoFloats-synthetic-BGC', 'ArgoFloats-reference']:
        try:
            # Set the dataset ID
            erddap.dataset_id = datasetID
            
            # Fetch data as a 2D dataframe with timeout handling
            dataset_df = erddap.to_pandas(
                requests_kwargs={'timeout': timeout_seconds}
            )
            
            # Process data (e.g., print the columns)
            print(dataset_df.columns)

        
        except requests.exceptions.Timeout:
            print(f"Timeout error for {datasetID}: The request took longer than {timeout_seconds} seconds.")

        
        except Exception as e:
            print(f"There was an error for {datasetID}: {e}")

there are 53 datasets available via ARGO ERDDAP
Processing dataset: allDatasets
Index(['datasetID', 'accessible', 'institution', 'dataStructure',
       'cdm_data_type', 'class', 'title', 'minLongitude (degrees_east)',
       'maxLongitude (degrees_east)', 'longitudeSpacing (degrees_east)',
       'minLatitude (degrees_north)', 'maxLatitude (degrees_north)',
       'latitudeSpacing (degrees_north)', 'minAltitude (m)', 'maxAltitude (m)',
       'minTime (UTC)', 'maxTime (UTC)', 'timeSpacing (seconds)', 'griddap',
       'subset', 'tabledap', 'MakeAGraph', 'sos', 'wcs', 'wms', 'files',
       'fgdc', 'iso19115', 'metadata', 'sourceUrl', 'infoUrl', 'rss', 'email',
       'testOutOfDate', 'outOfDate', 'summary'],
      dtype='object')
Processing dataset: OACP-Argo-Global
There was an error for OACP-Argo-Global: The read operation timed out
Processing dataset: ArgoFloats
Processing dataset: ArgoFloats-synthetic-BGC
Processing dataset: ArgoFloats-reference
Processing dataset: ArgoFloats-inde

KeyboardInterrupt: 

In [46]:
try:
    #connect to server 
    erddap = ERDDAP(server="https://erddap.ifremer.fr/erddap",protocol="tabledap")
    #Set a dataset ID
    erddap.dataset_id = 'drifter_hourly_qc'
    #Get data (as 2D dataframe)
    dataset_df = erddap.to_pandas()
    print(dataset_df.columns)
except Exception as e:
    print(f'There was an error for {datasetID}: {e}')

KeyboardInterrupt: 

In [ ]:
### code te clean up

In [ ]:
# other metadata (~written metadata, intended for human reading?) available with df.summary
argoERDDAP_allDatasets_df.summary

In [ ]:
# argoFloats dataset lists file not actual data --> need extra step to retrieve data

# getting an actual data file
base_url = "https://data-argo.ifremer.fr/dac/" #need to look up and hardcode 
filepath = argoERDDAP_argoFloats_idx_df.loc[0, 'file']
filename = filepath.split('/')[-1]
url = base_url+filepath

response = requests.get(url)
with open(filename, 'wb') as file:
    file.write(response.content)

dataset = nc.Dataset(filename, 'r')
dataset